In [ ]:
!pip install pickle


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [ ]:
import string
import re
from pickle import dump, load
from unicodedata import normalize
import pickle


from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# split a loaded document into sentences
def to_sentences(doc):
	return doc.strip().split('\n')

# clean a list of lines
def clean_lines(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for line in lines:
		# normalize unicode characters
		line = normalize('NFD', line).encode('ascii', 'ignore')
		line = line.decode('UTF-8')
		# tokenize on white space
		line = line.split()
		# convert to lower case
		line = [word.lower() for word in line]
		# remove punctuation from each token
		line = [word.translate(table) for word in line]
		# remove non-printable chars form each token
		line = [re_print.sub('', w) for w in line]
		# remove tokens with numbers in them
		line = [word for word in line if word.isalpha()]
		# store as string
		cleaned.append(' '.join(line))
	return cleaned

# save a list of clean sentences to file
def save_clean_sentences(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load English data
filename = 'training/europarl-v7.fr-en.en'
doc = load_doc(filename)
sentences = to_sentences(doc)
sentences = clean_lines(sentences)
save_clean_sentences(sentences, 'english.pkl')
# spot check
for i in range(10):
	print(sentences[i])

# load French data
filename = 'training/europarl-v7.fr-en.fr'
doc = load_doc(filename)
sentences = to_sentences(doc)
sentences = clean_lines(sentences)
save_clean_sentences(sentences, 'french.pkl')
# spot check
for i in range(1):
	print(sentences[i])

In [ ]:

with open('english.pkl', 'rb') as f:
    eng_voc = pickle.load(f)
eng_voc

In [ ]:
with open('french.pkl', 'rb') as f:
    fr_voc = pickle.load(f)
fr_voc

NameError: name 'pickle' is not defined

In [ ]:
print(len(eng_voc))
len(fr_voc)

2007723


2007723

In [ ]:
data = pd.DataFrame(zip(eng_voc, fr_voc), columns = ['English', 'French'])
data

,English,French
0,resumption of the session,reprise de la session
1,i declare resumed the session of the european ...,je declare reprise la session du parlement eur...
2,although as you will have seen the dreaded mil...,comme vous avez pu le constater le grand bogue...
3,you have requested a debate on this subject in...,vous avez souhaite un debat a ce sujet dans le...
4,in the meantime i should like to observe a min...,en attendant je souhaiterais comme un certain ...
...,...,...
2007718,i would also like although they are absent to ...,je me permettrai meme bien quils soient absent...
2007719,i am not going to reopen the millennium or not...,je ne rouvrirai pas le debat sur le millenaire...
2007720,adjournment of the session,interruption de la session
2007721,i declare the session of the european parliame...,je declare interrompue la session du parlement...


In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

--2022-03-17 23:57:07--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 

18.67.76.99, 18.67.76.37, 18.67.76.42, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.67.76.99|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip              0%[                    ]       0  --.-KB/s               

data.zip            100%[===================>]   2.75M  --.-KB/s    in 0.1s    

2022-03-17 23:57:07 (20.9 MB/s) - ‘data.zip’ saved [2882130/2882130]



Archive:  data.zip
   creating: data/


  inflating: data/eng-fra.txt        

   creating: data/names/


  inflating: data/names/Arabic.txt   
  inflating: data/names/Chinese.txt  
  inflating: data/names/Czech.txt    
  inflating: data/names/Dutch.txt    
  inflating: data/names/English.txt  
  inflating: data/names/French.txt   
  inflating: data/names/German.txt   
  inflating: data/names/Greek.txt    
  inflating: data/names/Irish.txt    
  inflating: data/names/Italian.txt  
  inflating: data/names/Japanese.txt  
  inflating: data/names/Korean.txt   
  inflating: data/names/Polish.txt   
  inflating: data/names/Portuguese.txt  
  inflating: data/names/Russian.txt  
  inflating: data/names/Scottish.txt  
  inflating: data/names/Spanish.txt  
  inflating: data/names/Vietnamese.txt  


In [ ]:
#the original dataset for the tutorial
data2 = pd.read_csv('data/eng-fra.txt', '\t', names = ['English', 'French'])

data2

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


,English,French
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !
...,...,...
135837,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
135838,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
135839,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
135840,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [ ]:
#concat the two dfs
data = pd.concat([data,data2], ignore_index= True, axis = 0)

data

,English,French
0,resumption of the session,reprise de la session
1,i declare resumed the session of the european ...,je declare reprise la session du parlement eur...
2,although as you will have seen the dreaded mil...,comme vous avez pu le constater le grand bogue...
3,you have requested a debate on this subject in...,vous avez souhaite un debat a ce sujet dans le...
4,in the meantime i should like to observe a min...,en attendant je souhaiterais comme un certain ...
...,...,...
2279402,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
2279403,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
2279404,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
2279405,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [ ]:
data.to_csv('eng-fra.txt', sep = "\t")

In [ ]:
data

,English,French
0,resumption of the session,reprise de la session
1,i declare resumed the session of the european ...,je declare reprise la session du parlement eur...
2,although as you will have seen the dreaded mil...,comme vous avez pu le constater le grand bogue...
3,you have requested a debate on this subject in...,vous avez souhaite un debat a ce sujet dans le...
4,in the meantime i should like to observe a min...,en attendant je souhaiterais comme un certain ...
...,...,...
2279402,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
2279403,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
2279404,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
2279405,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...


In [ ]:
#find most common initial bigrams to feed into the english prefixes
import pandas as pd
# df2 = pd.read_csv('eng-fra2.txt', '\t')
lst = []
for i in df2['Go.']:
    lst.append(" ".join(i.split(' ')[:2]))
df2['commons english'] = lst

In [ ]:
df2['commons english'].value_counts()[:50].index

Index(['I don't', 'Do you', 'I want', 'Are you', 'I have', 'I think',
       'I can't', 'I was', 'He is', 'I'm not', 'This is', 'I just', 'I didn't',
       'I am', 'I thought', 'I know', 'Tom is', 'I had', 'Did you', 'Have you',
       'Can you', 'He was', 'You don't', 'I'd like', 'It was', 'You should',
       'Would you', 'I like', 'It is', 'She is', 'You can't', 'He has',
       'What do', 'If you', 'I need', 'No one', 'You are', 'You have',
       'I feel', 'I really', 'Why don't', 'I hope', 'I will', 'We have',
       'You're not', 'You're very', 'She was', 'I love', 'You must', 'I can'],
      dtype='object')